#### 1. Import pandas library

In [7]:
import pandas as pd

#### 2. Import pymysql and sqlalchemy as you have learnt in the lesson of importing/exporting data 


In [8]:
from sqlalchemy import create_engine
import pymysql

#### 3. Create a mysql engine to set the connection to the server. Check the connection details in [this link](https://relational.fit.cvut.cz/search?tableCount%5B%5D=0-10&tableCount%5B%5D=10-30&dataType%5B%5D=Numeric&databaseSize%5B%5D=KB&databaseSize%5B%5D=MB)

In [9]:
driver = 'mysql+pymysql:'
user = 'guest'
password = 'relational'
ip = 'relational.fit.cvut.cz'
database = 'stats'

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

#### 4. Import the users table 

In [10]:
users = pd.read_sql('users',con=engine)

#### 5. Rename Id column to userId

In [11]:
users = users.rename(columns={'Id':'userId'})

#### 6. Import the posts table. 

In [12]:
posts = pd.read_sql('posts', con=engine)

#### 7. Rename Id column to postId and OwnerUserId to userId

In [13]:
posts = posts.rename(columns={'Id':'postId','OwnerUserId':'userId'})

#### 8. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userID,ViewCount,CommentCount

In [14]:
users_selection = users[['userId', 'Reputation', 'Views', 'UpVotes', 'DownVotes']]
posts_selection = posts[['postId', 'Score', 'userId', 'ViewCount', 'CommentCount']]

#### 8. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [15]:
user_posts = users_selection.merge(posts_selection, how='left', on='userId')

#### 9. How many missing values do you have in your merged dataframe? On which columns?

In [16]:
user_posts.isna().sum()

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId          18342
Score           18342
ViewCount       66738
CommentCount    18342
dtype: int64

#### 10. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [17]:
user_posts = user_posts.drop(columns='ViewCount')

In [18]:
user_posts = user_posts[user_posts['postId'].notna()]

In [19]:
user_posts.isna().sum()

userId          0
Reputation      0
Views           0
UpVotes         0
DownVotes       0
postId          0
Score           0
CommentCount    0
dtype: int64

#### 11. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [20]:
user_posts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90584 entries, 0 to 108924
Data columns (total 8 columns):
userId          90584 non-null int64
Reputation      90584 non-null int64
Views           90584 non-null int64
UpVotes         90584 non-null int64
DownVotes       90584 non-null int64
postId          90584 non-null float64
Score           90584 non-null float64
CommentCount    90584 non-null float64
dtypes: float64(3), int64(5)
memory usage: 6.2 MB


In [ ]:
#None, all of them are numeric